**Date** 24/11/2025
**Author** Jordi Torres
Core code based on work by professor Gaillard. 

**EXERCISE 2** Ayagari + employment status + search effort and ex destruction rate

In [19]:
#libraries 

using Printf;
using Roots;
using Plots;
using Distributions;
using LambertW;

#wider page display
display("text/html", "<style>.container { width:100% !important; }</style>")

In [20]:
const size_k 	= 1000;    #number of grid points     
const size_m 	= 7;       #number of productivity classes

const size_e =2 ; #added!

const kmin 		= 0.0;
const kmax 		= 1700.0;

betapar 	= 0.96;
const alphapar 	= 0.36;
const deltapar 	= 0.08;
const rhopar 	= 3.0;
const labor 	= 1.0219882;

const epsV    = 1.0e-6
const epsdist = 1.0e-8
const epsK    = 1.0e-6   # for debugging


## grid constants
const scale1 = 1.6;
const grmin = (kmin/scale1)-1.0;
const exponen=log((kmax/scale1)-grmin)/(size_k-1);

const sstates = [exp(-0.600000000000000)	  exp(-0.400000000000000)	  exp(-0.200000000000000)	  exp(0.000000000000000)	   exp(0.200000000000000)	   exp(0.400000000000000)	   exp(0.600000000000000)];

const strans = [	
    0.046746218637144	   0.217937777267117	   0.397822606398702	   0.266386738072197	   0.065169922261456	   0.005754191945237	   0.000182545418147;	
	0.023199661746751	   0.149524091076020	   0.369020347246402	   0.333823905199677	   0.110578117872631	   0.013276146769082	   0.000577730089437;	
	0.010548958644399	   0.093657511915497	   0.312761268311836	   0.382193227897354	   0.171253064028981	   0.027919224002876	   0.001666745199056;	
	0.004387354018187	   0.053538402796357	   0.242163972572887	   0.399820541225137	   0.242163972572887	   0.053538402796357	   0.004387354018187;	
	0.001666745199056	   0.027919224002876	   0.171253064028981	   0.382193227897354	   0.312761268311837	   0.093657511915497	   0.010548958644399;	
	0.000577730089436	   0.013276146769082	   0.110578117872631	   0.333823905199677	   0.369020347246403	   0.149524091076020	   0.023199661746751;	
	0.000182545418147	   0.005754191945237	   0.065169922261456	   0.266386738072197	   0.397822606398702	   0.217937777267117	   0.046746218637144];

#extension:

 b= 0.35
 tau_wage=0.25

const ν= 0.04

etrans = [
	0.04 0.96
]

emplo = [0.0, 1.0]

2-element Vector{Float64}:
 0.0
 1.0

In [21]:
function utility(cons::Float64)
    cons_adj = max(cons, 1.0e-8)      
    return (cons_adj^(1.0-rhopar)) / (1.0 - rhopar)
end


function invmargutil(x::Float64)
    x_adj = max(x, 1.0e-12)     # ensure strictly positive
    return x_adj^(-1.0/rhopar)
end

##add!!!!!

invmargutil (generic function with 1 method)

In [22]:
function inter1d(x1::Float64,y1::Float64,y2::Float64) 
    return ((1.0-(x1))*(y1)+(x1)*(y2))
end

function getwage(rrate::Float64)
    return (1.0-alphapar)*(alphapar/(rrate+deltapar))^(alphapar/(1.0-alphapar));
end

function getlevel(igrid::Int64)
    #careful this function assumes that grid starts at 0
    return ( scale1*(exp(exponen*(igrid))+grmin) );
end

function getgrid(x::Float64)
    #careful because indexes start at 1 in Julia, a +1.0 was added
    return (log((x)/scale1-grmin)/exponen)+1.0;
end


function get_x(EV::Float64, κ::Float64, β::Float64)
    x_star=0.0
    if EV>=0 ##assumption on ties???
     x_star= (1/κ)*lambertw(κ^2.0 *β*EV)
    end
    return x_star
end

function get_xi(x_star::Float64, κ::Float64)
    return  1- exp(-κ*x_star)
end

get_xi (generic function with 1 method)

In [23]:
function nderiv(val1::Float64,val2::Float64,val3::Float64,x1::Float64,x2::Float64,x3::Float64) 
    return ((1.0 - (x3 - x2)/(x3 - x1))*((val3 - val2)/(x3-x2)) + ((x3 - x2)/(x3 - x1))*((val2 - val1)/(x2-x1)));
end

nderiv (generic function with 1 method)

In [24]:
function POLICY_we(valueF::Array{Float64,3},dvalueF::Array{Float64,3},save::Array{Float64,3},wagerate::Float64,rrate::Float64, grid::Array{Float64}, betapar::Float64, tau_wage::Float64, b::Float64, κ::Float64, β::Float64, ν::Float64)

    aendo  = zeros(Float64, size_k, size_m, size_e);
    xendo = zeros(Float64, size_k, size_m, size_e);
    EVendo = zeros(Float64, size_k, size_m, size_e);  
    etrans_e = zeros(Float64, size_k, size_m, size_e, size_e);
    xpol= zeros(Float64, size_k, size_m, size_e);
    iter=0;
    

    prob_e=zeros(2,2) ##where do i construct here

    critV=10000.0;

    while critV>epsV

        # we need copy to make a separate object
        valueFnew = copy(valueF);

        # COMPUTE EXPECTED VALUE / DERIVATIVE / ENDOGENEOUS GRID 
      @inbounds for iemplo::Int64 in 1:1:(size_e)  

            @inbounds Threads.@threads  for iprod::Int64 in 1:1:(size_m) #this is the s_t
                @inbounds for igrid::Int64 in 1:1:(size_k) #this is a' or a_t+1
                   
                                        
                        if emplo[iemplo] == 1
                            # === EMPLOYED TODAY (e = 1 in your emplo vector, iemplo = 2 if you use [0,1]) ===
                            EV_0  = 0.0
                            EV_1  = 0.0
                            dEV_0 = 0.0
                            dEV_1 = 0.0

                            @inbounds for iprodnext::Int64 in 1:1:size_m
                                EV_0  += strans[iprod, iprodnext] * valueF[igrid, iprodnext, 1]  # unemployed next
                                EV_1  += strans[iprod, iprodnext] * valueF[igrid, iprodnext, 2]  # employed next
                                dEV_0 += strans[iprod, iprodnext] * dvalueF[igrid, iprodnext, 1]
                                dEV_1 += strans[iprod, iprodnext] * dvalueF[igrid, iprodnext, 2]
                            end

                            # Exogenous separation: ν
                            tempnext  = ν * EV_0 + (1.0 - ν) * EV_1
                            dtempnext = ν * dEV_0 + (1.0 - ν) * dEV_1

                            # Store transition probs
                            etrans_e[igrid, iprod, 2, 1] = ν
                            etrans_e[igrid, iprod, 2, 2] = 1.0 - ν

                            xendo[igrid, iprod, iemplo] = 0.0
                        else
                            # === UNEMPLOYED TODAY (your existing branch, with ΔEV, x_star, delta_par) ===
                            EV_0  = 0.0
                            EV_1  = 0.0
                            dEV_0 = 0.0
                            dEV_1 = 0.0

                            @inbounds for iprodnext::Int64 in 1:1:size_m
                                EV_0  += strans[iprod, iprodnext] * valueF[igrid, iprodnext, 1]
                                EV_1  += strans[iprod, iprodnext] * valueF[igrid, iprodnext, 2]
                                dEV_0 += strans[iprod, iprodnext] * dvalueF[igrid, iprodnext, 1]
                                dEV_1 += strans[iprod, iprodnext] * dvalueF[igrid, iprodnext, 2]
                            end

                            ΔEV = EV_1 - EV_0

                            x_star = get_x(ΔEV, κ, β)
                            xendo[igrid, iprod, iemplo] = x_star

                            delta_par = get_xi(x_star, κ)

                            # store endogenous transition probs
                            etrans_e[igrid, iprod, 1, 1] = 1.0 - delta_par
                            etrans_e[igrid, iprod, 1, 2] = delta_par

                            tempnext  = (1.0 - delta_par) * EV_0 + delta_par * EV_1
                            dtempnext = (1.0 - delta_par) * dEV_0 + delta_par * dEV_1
                        end



                    # computing endogenous grid using the first order condition
                    aendo[igrid,iprod, iemplo]  = (invmargutil(betapar*dtempnext) + grid[igrid] - wagerate * (emplo[iemplo]+ b*(1-emplo[iemplo]))*(1-tau_wage)* sstates[iprod])/(1.0 + rrate);
                    EVendo[igrid,iprod, iemplo] = tempnext;

                end
            end
       end

        @inbounds for iemplo::Int64 in 1:1:(size_e) 

            # RESCALING STEP
            @inbounds Threads.@threads for iprod::Int64 in 1:1:(size_m) #current productivity state
                ilast=1;
                @inbounds for igrid::Int64 in 1:1:(size_k) #current level of assets 
                    
                    # build cohexo for simplicity
                    cohexo = (1.0 + rrate) * grid[igrid] + wagerate * sstates[iprod]*(emplo[iemplo]+ b*(1-emplo[iemplo]))*(1-tau_wage);


                    # case 1: borrowing constrained case
                    if grid[igrid] <= aendo[1,iprod, iemplo]

                        save[igrid,iprod, iemplo]   = grid[1];
                        evfnext = EVendo[1,iprod, iemplo];

                        xpol[igrid,iprod,iemplo] = xendo[1,iprod,iemplo]

                    else ## otherwise I am either in case 2 or case 3

                        ## This is an improvement because we do not need to restart from scratch (due to monotonicity). 
                        ## We start with the previous grid point solution.
                        itest=ilast;  

                        # case 2: extrapolation case
                        if grid[igrid] > aendo[size_k,iprod, iemplo]
                            ## extrapolation so vfweight > 1
                            vfweight = (grid[igrid]-aendo[size_k-1,iprod, iemplo])/(aendo[size_k,iprod, iemplo]-aendo[size_k-1,iprod, iemplo]);
                            igridL   = size_k-1;
                            igridH   = size_k;
                        else

                            #finding interpolating points on cohendo
                            while ((itest<(size_k+1)&&(grid[igrid]>aendo[itest,iprod, iemplo])))
                                itest+=1;
                            end
                            
                            ## some sanity checks
                            if itest>(size_k+1)
                                @printf("itest>(size_k+1)");
                                exit()
                            end

                            # standard interior case
                            if itest==1
                                @printf("itest==1");
                                exit()
                            end

                            vfweight = (grid[igrid]-aendo[itest-1,iprod, iemplo])/(aendo[itest,iprod, iemplo]-aendo[itest-1,iprod, iemplo]);
                            igridL   = itest-1;
                            igridH   = itest;

                        end

                        ## linear interpolation of the value function and saving function using above weights and indexes
                        evfnext = inter1d(vfweight,EVendo[igridL,iprod, iemplo],EVendo[igridH,iprod, iemplo]);
                        save[igrid,iprod, iemplo] = inter1d(vfweight,grid[igridL],grid[igridH]);    
                        
                        xpol[igrid,iprod, iemplo]= inter1d(vfweight, xendo[igridL,iprod, iemplo], xendo[igridH,iprod, iemplo]) ##NOTE: new interpolation added! To revise. This is over the other policy function. 

                        ilast = min(size_k-1,itest);

                    end

                    ## construct the new VF

                    valueF[igrid,iprod, iemplo] = utility(cohexo-save[igrid,iprod, iemplo]) -(1.0-emplo[iemplo])*((xpol[igrid,iprod, iemplo]^2)/2) + betapar * evfnext;
 ;
                    #this we should condition on employment and unemployment status!!!! We should get this effort from extrapolation!!!!!

                end
            end
        end 





        # computing new derivatives and convergence
        critV=0.0;
        @inbounds for iemplo::Int64 in 1:1:(size_e)
            @inbounds Threads.@threads for iprod::Int64 in 1:1:(size_m)
                @inbounds for igrid::Int64 in 1:1:(size_k)
            
                    # interior deriv
                    if (igrid>=3)
                        dvalueF[igrid-1,iprod, iemplo]=nderiv(valueF[igrid-2,iprod, iemplo],valueF[igrid-1,iprod, iemplo],valueF[igrid,iprod, iemplo],grid[igrid-2],grid[igrid-1],grid[igrid]);
                    end

                    critV=max(critV,abs(valueF[igrid,iprod, iemplo]-valueFnew[igrid,iprod, iemplo]));

                    # left corner
                    dvalueF[1,iprod, iemplo]=(valueF[2,iprod, iemplo]-valueF[1,iprod, iemplo])/(grid[2]-grid[1]);

                    # right corner
                    dvalueF[size_k,iprod, iemplo]=(valueF[size_k,iprod, iemplo]-valueF[size_k-1,iprod, iemplo])/(grid[size_k]-grid[size_k-1]);
        
                end
            end
        end 
        iter+=1;
         #@printf("CNVG %4d\t%20.15f\n",iter,critV);

    end
    
    return prob_e=etrans_e
end

POLICY_we (generic function with 1 method)

In [25]:
function SIMULATION_we(save::Array{Float64,3},dist::Array{Float64,3},grid::Array{Float64}, etrans_e::Array{Float64,4})

    distold = zeros(Float64, size_k, size_m, size_e);
    
    critdist=1.0;
    while critdist>epsdist

        distold=copy(dist);
        fill!(dist, 0.0);

        for iemplo in 1:size_e
        for iprod in 1:size_m
            for igrid in 1:size_k
            if distold[igrid,iprod,iemplo] > 0.0

                isave  = min(floor(Int64, getgrid(save[igrid,iprod,iemplo])), size_k-1)
                weight = (save[igrid,iprod,iemplo] - grid[isave]) / (grid[isave+1] - grid[isave])

                @inbounds for iemplonext in 1:size_e
                    prob_e = etrans_e[igrid, iprod, iemplo, iemplonext]

                    @inbounds for iprodnext in 1:size_m
                        prob = strans[iprod, iprodnext] * prob_e

                        dist[isave, iprodnext, iemplonext]               += (1.0 - weight) * prob * distold[igrid,iprod,iemplo]
                        dist[min(isave+1,size_k), iprodnext, iemplonext] += weight         * prob * distold[igrid,iprod,iemplo]
                    end
                end
            end
            end
        end
        end





        # check convergence
        critdist=0.0;
        distverif = 0.0;
        @inbounds for iemplo::Int64 in 1:1:(size_e)
            @inbounds for iprod::Int64 in 1:1:(size_m)
                @inbounds for igrid::Int64 in 1:1:(size_k)
                    critdist=(max(critdist,abs(dist[igrid,iprod,iemplo]-distold[igrid,iprod, iemplo]))) ;
                    distverif+=dist[igrid,iprod, iemplo];
                end
            end
        end 
        # @printf("SIM %20.15f\t%20.15f\n", critdist, distverif);
    end




    # computing aggregates

    #capital
    capitalout=0.0;
    @inbounds for iemplo::Int64 in 1:1:(size_e)
        @inbounds for iprod::Int64 in 1:1:(size_m)
            @inbounds for igrid::Int64 in 1:1:(size_k)
                capitalout+=dist[igrid,iprod, iemplo]*grid[igrid];
                # @printf("%d\t%d\t%20.15f\n",iprod,igrid,dist[igrid,iprod]);
            end
        end
    end 
    #this we should condition on the status of employment, unemployment->endogeneous NOTE
  laborout=0.0;
    @inbounds for iemplo::Int64 in 1:1:(size_e)
        @inbounds for iprod::Int64 in 1:1:(size_m)
            @inbounds for igrid::Int64 in 1:1:(size_k)
                laborout+=dist[igrid,iprod, iemplo]*sstates[iprod]*(emplo[iemplo]);
                # @printf("%d\t%d\t%20.15f\n",iprod,igrid,dist[igrid,iprod]);

                #distribution should change based on level of effort now
            end
        end
    end 


    # println(capitalout);
    # read(stdin, Char);

    return(capitalout, laborout);


end

SIMULATION_we (generic function with 1 method)

In [26]:
const relaxsK 	= 0.05;

# functions for the equilibrium wage.
function getwage_eq(LABOR::Float64,CAPITAL::Float64)
    return (1.0-alphapar) * getGDP(LABOR,CAPITAL)/LABOR;
end

# function for the equilibrium interest rate. 
function getrate_eq(LABOR::Float64,CAPITAL::Float64)
    return (alphapar) * getGDP(LABOR,CAPITAL)/CAPITAL - deltapar;
end

 # functions for the equilibrium.
function getGDP(LABOR::Float64,CAPITAL::Float64)
    return (LABOR^(1.0-alphapar)) * (CAPITAL^(alphapar));
end

getGDP (generic function with 1 method)

In [27]:

function main_ge(betapar::Float64,impose_budget_balance::Bool, b::Float64, κ::Float64, β::Float64, ν::Float64)


    valueF  = zeros(Float64, size_k, size_m, size_e);
    dvalueF = zeros(Float64, size_k, size_m, size_e);
    save    = zeros(Float64, size_k, size_m, size_e);
    dist    = zeros(Float64, size_k, size_m, size_e);
    grid    = Array{Float64}(undef, size_k);

    # nth = Threads.nthreads(); #print number of threads
    @printf("Aiyagari1994 using %d threads\n",Threads.nthreads());
    # read(stdin, Char);




 # tabulating main grid
    for i::Int64 in 0:1:(size_k-1)
        grid[i+1]=getlevel(i);
        ##@printf("%d\t%20.15f\t%20.15f\n",i+1,getlevel(i),getgrid(getlevel(i)) );
    end



 # initial prices, gdp!!!
    LABOR   = 0.4
    CAPITAL = 2.60
    gdp= 0
    fixed_tau=0.25
    β = 0.96     # baseline  
    κ = 0.04     # used in most papers  


    rrate    = getrate_eq(LABOR,CAPITAL);
    wagerate = getwage_eq(LABOR,CAPITAL);

    @printf("%f %f\n",rrate,wagerate);

 # distribution init
    dist .= 0.0
    dist[1, ceil(Int, size_m/2), 2] = 1.0   # asset zero, middle s, employed


    # === 1. Initialize value function ===
    for iemplo in 1:size_e
        for iprod in 1:size_m
            for igrid in 1:size_k
                a = grid[igrid]

                # income if employed (e=1) or unemployed (e=0) with benefit b
                inc = wagerate * sstates[iprod] *
                    (emplo[iemplo] + b * (1.0 - emplo[iemplo])) *
                    (1.0 - fixed_tau)

                # consume everything today, no effort cost (x=0)
                c = (1.0 + rrate) * a + inc

                # clamp c > 0 inside utility (your utility() already does)
                valueF[igrid, iprod, iemplo] = utility(c) / (1.0 - betapar)
            end
        end
    end

    # === 2. Compute initial derivatives ===
    for iemplo in 1:size_e
        for iprod in 1:size_m

            # left
            dvalueF[1, iprod, iemplo] =
                (valueF[2,iprod,iemplo] - valueF[1,iprod,iemplo])/(grid[2]-grid[1])

            # interior
            for igrid in 3:size_k
                dvalueF[igrid-1, iprod, iemplo] =
                    nderiv(valueF[igrid-2,iprod,iemplo],
                        valueF[igrid-1,iprod,iemplo],
                        valueF[igrid,iprod,iemplo],
                        grid[igrid-2],grid[igrid-1],grid[igrid])
            end

            # right
            dvalueF[size_k, iprod, iemplo] =
                (valueF[size_k,iprod,iemplo] -
                valueF[size_k-1,iprod,iemplo])/(grid[end]-grid[end-1])
        end
    end

   @time begin
    ge_iter = 0
    max_ge_iter = 80

    critK = 1.0

    while critK > epsK && ge_iter < max_ge_iter
        ge_iter += 1

        t_iter_start = time()

        if (betapar*(1.0+rrate)>=1.0)
            println("Beta condition is not respected ",rrate," ",(betapar*(1.0+rrate)));
            exit();
        end



        ### --- TIMER: POLICY STEP ---
        t_policy_start = time()
        p_trans=POLICY_we(valueF, dvalueF, save, wagerate, rrate, grid, betapar, fixed_tau, b,κ, β, ν)
        t_policy = time() - t_policy_start


        ### --- TIMER: SIMULATION STEP ---
        t_sim_start = time()
        SIMULATION_we(save, dist, grid, p_trans)
        t_sim = time() - t_sim_start

        ### --- COMPUTE AGGREGATES ---

        # === GOVERNMENT BUDGET CONSTRAINT ===
            if impose_budget_balance ##This condition is set if we want to impose budget balance. 
                L_emp   = 0.0
                L_unemp = 0.0

                for iemplo in 1:size_e
                    for iprod in 1:size_m
                        for igrid in 1:size_k
                            mass = dist[igrid,iprod,iemplo]
                            s    = sstates[iprod]
                            e    = emplo[iemplo]

                            if e == 1
                                L_emp   += mass * s
                            else
                                L_unemp += mass * s
                            end
                        end
                    end
                end

                fixed_tau = b * L_unemp / L_emp
            end




        capitalnew = 0.0
        labornew   = 0.0

        @inbounds for iemplo::Int in 1:size_e
            @inbounds for iprod::Int in 1:size_m
                @inbounds for igrid::Int in 1:size_k
                    capitalnew += dist[igrid,iprod,iemplo] * grid[igrid]
                    labornew   += dist[igrid,iprod,iemplo] * emplo[iemplo] * sstates[iprod]
                    #this now depends on employed/unemployed division. 
                end
            end
        end

        # compute convergence
        wwage0 = wagerate
        rrate0 = rrate
        critK = abs((capitalnew - CAPITAL) / CAPITAL)

        # relaxation update
        CAPITAL = relaxsK * capitalnew + (1.0 - relaxsK) * CAPITAL
        LABOR   = relaxsK * labornew   + (1.0 - relaxsK) * LABOR

        # new prices
        rrate    = getrate_eq(LABOR, CAPITAL)
        wagerate = getwage_eq(LABOR, CAPITAL)
        gdp      = getGDP(LABOR, CAPITAL)



        t_iter = time() - t_iter_start

        ### === PRINT TIMERS AND RESULTS ===
        @printf("GE %3d | critK = %1.6e | t_policy = %4.3f s | t_sim = %4.3f s | t_iter = %4.3f s\n",
                ge_iter, critK, t_policy, t_sim, t_iter)

        @printf("         r=[%5f→%5f],  K=[%5f→%5f],  L=[%5f→%5f],  KY=%5f\n\n",
                rrate0, rrate, CAPITAL, capitalnew, LABOR, labornew, CAPITAL/gdp)

    end
 end



    distI = zeros(Float64, size_k);

    for iemplo::Int64 in 1:1:(size_e)
        for iprod::Int64 in 1:1:(size_m)
            for igrid::Int64 in 1:1:(size_k)
                distI[igrid] += dist[igrid,iprod, iemplo];
            end
        end
    end 


    welfare = 0.0
        for iemplo in 1:size_e
            for iprod in 1:size_m
                for igrid in 1:size_k
                    welfare += dist[igrid, iprod, iemplo] * valueF[igrid, iprod, iemplo]
                end
            end
        end

  return CAPITAL, LABOR, gdp,  dist, valueF, welfare, fixed_tau
end



main_ge (generic function with 1 method)

In [28]:
b_CAPITAL, b_LABOR, b_gdp,  dist, valueF, b_welfare, fixed_tau= main_ge(betapar, false, b, κ, β, ν);

Aiyagari1994 using 1 threads
0.028652 1.255535
GE   1 | critK = 2.702217e+00 | t_policy = 2.308 s | t_sim = 0.110 s | t_iter = 2.418 s
         r=[0.028652→0.020036],  K=[2.951288→9.625763],  L=[0.399053→0.381061],  KY=3.598718

GE   2 | critK = 1.114273e+00 | t_policy = 1.705 s | t_sim = 0.074 s | t_iter = 1.780 s
         r=[0.020036→0.017046],  K=[3.115715→6.239829],  L=[0.401782→0.453628],  KY=3.709567

GE   3 | critK = 7.765081e-01 | t_policy = 1.621 s | t_sim = 0.052 s | t_iter = 1.673 s
         r=[0.017046→0.015198],  K=[3.236684→5.535093],  L=[0.405028→0.466704],  KY=3.781584

GE   4 | critK = 5.980085e-01 | t_policy = 1.550 s | t_sim = 0.042 s | t_iter = 1.593 s
         r=[0.015198→0.013919],  K=[3.333462→5.172249],  L=[0.408415→0.472775],  KY=3.833080

GE   5 | critK = 4.841896e-01 | t_policy = 1.407 s | t_sim = 0.037 s | t_iter = 1.444 s
         r=[0.013919→0.012983],  K=[3.414164→4.947490],  L=[0.411806→0.476229],  KY=3.871673

GE   6 | critK = 4.044322e-01 | t_policy = 